# Evaluation

> Evaluation of DistCurve on several datsets

In [ ]:
from easydict import EasyDict
from dist_curve.curve_constructor import makeCurve
from dist_curve.model import getTrainedEstimator
import pickle
# model = getTrainedEstimator(weights_path="/data/dzeiberg/ClassPriorEstimation/model.hdf5")
model = getTrainedEstimator(weights_path="/home/dz/research/ClassPriorEstimation/model.hdf5")

In [ ]:
from glob import glob
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def getDSAbsErrs(ds):
    absErrs = []
    for inst in ds.instances:
        curve = makeCurve(ds.instances[0].posScores.reshape((-1,1)),
                          ds.instances[0].unlabeledScores.reshape((-1,1))).reshape((1,-1))
        alphaHat = model.predict(curve/curve.sum())
        absErrs.append(np.abs(inst.alpha - alphaHat))
    return absErrs

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hiddenLayer0 (Dense)         (None, 2048)              206848    
_________________________________________________________________
batch_normalization_6 (Batch (None, 2048)              8192      
_________________________________________________________________
activation_6 (Activation)    (None, 2048)              0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 2048)              0         
_________________________________________________________________
hiddenLayer1 (Dense)         (None, 1024)              2098176   
_________________________________________________________________
batch_normalization_7 (Batch (None, 1024)              4096      
_________________________________________________________________
activation_7 (Activation)    (None, 1024)             

In [ ]:
files = glob("/data/dzeiberg/ClassPriorEstimation/processedDatasets_partial/*.pkl")
absErrs= []
for file in tqdm(files,total=len(files)):
    name = file.split("/")[-1].replace(".pkl","")
    ds = EasyDict(pickle.load(open(file,"rb")))
    aes = getDSAbsErrs(ds)
    absErrs.append(aes)
    print(name, "{:.3f}".format(np.mean(aes)))

In [ ]:
print("Overall MAE: {:.3f}".format(np.mean(absErrs)))